In [134]:
import country_converter as coco
import requests
import geopandas as gpd
import pandas as pd

In [135]:
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get DoDo Data

In [37]:
url = "https://globalapi.dodopizza.com/api/v1/pizzerias/count"
r = requests.get(url)

In [38]:
country_codes = [country["countryCode"] for country in r.json()["countries"]]

In [39]:
dodo_dfs = []
for country_code in tqdm(country_codes, desc="Parsing Locations"):
    r = requests.get(f"https://publicapi.dodois.io/{country_code}/api/v1/unitinfo/all")
    locations = r.json()
    dodo_df = pd.DataFrame(locations)
    dodo_df["CountryCode"] = country_code
    dodo_dfs.append(dodo_df)

Parsing Locations:   0%|          | 0/24 [00:00<?, ?it/s]

In [40]:
dodo_df = pd.concat(dodo_dfs, ignore_index=True)

In [41]:
dodo_locations_df = dodo_df[~dodo_df["Location"].isnull()]

In [42]:
def build_point(row):
    try:
        lon = row["Location"]["Longitude"]
        lat = row["Location"]["Latitude"]
    except Exception as e:
        print(row["Location"])
    return Point(lon, lat)

In [88]:
dodo_locations_df["geometry"] = dodo_locations_df.apply(
    lambda row: build_point(row), axis=1
)

/tmp/ipykernel_33633/1505705900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dodo_locations_df['geometry'] = dodo_locations_df.apply(lambda row: build_point(row), axis=1)


In [89]:
dodo_gdf = gpd.GeoDataFrame(dodo_locations_df, crs=4326)

In [90]:
dodo_gdf = dodo_gdf.to_crs(3576)
dodo_gdf.to_file("data/dodo_locations.gpkg")

## Open World data

In [129]:
country_pop_df = pd.read_csv(
    "data/API_SP.POP.TOTL_DS2_en_csv_v2_87/API_SP.POP.TOTL_DS2_en_csv_v2_87.csv"
)

In [138]:
country_pop_df["iso_3166_1_alpha_2_codes"] = country_pop_df.apply(
    lambda row: coco.convert(names=[row["Country Code"]], to="ISO2"), axis=1
)

AFE not found in ISO3
AFW not found in ISO3
ARB not found in ISO3
CEB not found in ISO3
CHI not found in ISO3
CSS not found in ISO3
EAP not found in ISO3
EAR not found in ISO3
EAS not found in ISO3
ECA not found in ISO3
ECS not found in ISO3
EMU not found in ISO3
EUU not found in ISO3
FCS not found in ISO3
HIC not found in ISO3
HPC not found in ISO3
IBD not found in ISO3
IBT not found in ISO3
IDA not found in ISO3
IDB not found in ISO3
IDX not found in ISO3
INX not found in ISO3
LAC not found in ISO3
LCN not found in ISO3
LDC not found in ISO3
LIC not found in ISO3
LMC not found in ISO3
LMY not found in ISO3
LTE not found in ISO3
MEA not found in ISO3
MIC not found in ISO3
MNA not found in ISO3
NAC not found in ISO3
OED not found in ISO3
OSS not found in ISO3
PRE not found in ISO3
PSS not found in ISO3
PST not found in ISO3
SAS not found in ISO3
SSA not found in ISO3
SSF not found in ISO3
SST not found in ISO3
TEA not found in ISO3
TEC not found in ISO3
TLA not found in ISO3
TMN not fo

In [140]:
country_pop_df = country_pop_df[["iso_3166_1_alpha_2_codes", "2023"]]

In [141]:
world_boundaries = gpd.read_file("data/world_boundaries.gpkg")

In [142]:
world_boundaries_reproj = world_boundaries.to_crs(3576)
world_boundaries_reproj.to_file("data/world_boundaries_reproj.gpkg")

## Merge data

In [155]:
dodo_counts_df = pd.DataFrame(
    dodo_gdf.groupby("CountryCode").size(), columns=["DODOs"]
).reset_index()

In [156]:
dodo_counts_df["iso_3166_1_alpha_2_codes"] = dodo_counts_df["CountryCode"].str.upper()
dodo_counts_df = dodo_counts_df.merge(
    world_boundaries_reproj[["geometry", "iso_3166_1_alpha_2_codes"]],
    on="iso_3166_1_alpha_2_codes",
    how="left",
)

In [157]:
dodo_counts_population_gdf = dodo_counts_df.merge(
    country_pop_df, on="iso_3166_1_alpha_2_codes", how="left"
)

In [158]:
dodo_counts_population_gdf = dodo_counts_population_gdf.rename(
    columns={"iso_3166_1_alpha_2_codes": "COUNTRY"}
)
dodo_counts_population_gdf = dodo_counts_population_gdf[
    ["DODOs", "COUNTRY", "geometry", "2023"]
]

In [166]:
dodo_counts_population_gdf["per_100k"] = (
    dodo_counts_population_gdf["DODOs"] / (dodo_counts_population_gdf["2023"] / 100_000)
).round(decimals=2)
dodo_counts_population_gdf["per_1m"] = (
    dodo_counts_population_gdf["DODOs"]
    / (dodo_counts_population_gdf["2023"] / 1_000_000)
).round(decimals=2)

In [169]:
dodo_counts_population_gdf.sort_values("per_1m", ascending=False)

,DODOs,COUNTRY,geometry,2023,per_100k,per_1m
18,1211,RU,"MULTIPOLYGON (((3403040.93 -3796309.94, 340002...",143826130.0,0.84,8.42
12,148,KZ,"MULTIPOLYGON (((-206621.28 -4461148.288, -2090...",20330104.0,0.73,7.28
4,53,BY,"MULTIPOLYGON (((-3527768.256 -2185935.142, -35...",9178298.0,0.58,5.77
7,6,EE,"MULTIPOLYGON (((-3185560.803 -1351196.362, -31...",1370286.0,0.44,4.38
13,11,LT,"MULTIPOLYGON (((-3561985.219 -1366234.642, -35...",2871585.0,0.38,3.83
5,3,CY,"MULTIPOLYGON (((-4847973.754 -3284793.567, -48...",1344976.0,0.22,2.23
11,14,KG,"MULTIPOLYGON (((-877920.364 -5100677.016, -877...",7099750.0,0.20,1.97
1,5,AM,"MULTIPOLYGON (((-3790449.151 -3999941.976, -37...",2990900.0,0.17,1.67
0,15,AE,"MULTIPOLYGON (((-4077961.819 -5605599.919, -40...",10483751.0,0.14,1.43
19,3,SI,"MULTIPOLYGON (((-4539707.301 -1354024.532, -45...",2120461.0,0.14,1.41


In [118]:
dodo_counts_gdf = gpd.GeoDataFrame(dodo_counts_df, crs=3576)
dodo_counts_gdf.to_file("data/dodo_country_counts.gpkg")